# 模型推理 - 使用 QLoRA 微调后的 ChatGLM-6B

In [1]:
import torch
from transformers import AutoModel, AutoTokenizer, BitsAndBytesConfig

# 模型ID或本地路径
model_name_or_path = 'THUDM/chatglm3-6b'

In [2]:
_compute_dtype_map = {
    'fp32': torch.float32,
    'fp16': torch.float16,
    'bf16': torch.bfloat16
}

# QLoRA 量化配置
q_config = BitsAndBytesConfig(load_in_4bit=True,
                              bnb_4bit_quant_type='nf4',
                              bnb_4bit_use_double_quant=True,
                              bnb_4bit_compute_dtype=_compute_dtype_map['bf16'])
# 加载量化后模型
base_model = AutoModel.from_pretrained(model_name_or_path,
                                  quantization_config=q_config,
                                  device_map='auto',
                                  trust_remote_code=True)

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

In [3]:
base_model.requires_grad_(False)
base_model.eval()

ChatGLMForConditionalGeneration(
  (transformer): ChatGLMModel(
    (embedding): Embedding(
      (word_embeddings): Embedding(65024, 4096)
    )
    (rotary_pos_emb): RotaryEmbedding()
    (encoder): GLMTransformer(
      (layers): ModuleList(
        (0-27): 28 x GLMBlock(
          (input_layernorm): RMSNorm()
          (self_attention): SelfAttention(
            (query_key_value): Linear4bit(in_features=4096, out_features=4608, bias=True)
            (core_attention): CoreAttention(
              (attention_dropout): Dropout(p=0.0, inplace=False)
            )
            (dense): Linear4bit(in_features=4096, out_features=4096, bias=False)
          )
          (post_attention_layernorm): RMSNorm()
          (mlp): MLP(
            (dense_h_to_4h): Linear4bit(in_features=4096, out_features=27392, bias=False)
            (dense_4h_to_h): Linear4bit(in_features=13696, out_features=4096, bias=False)
          )
        )
      )
      (final_layernorm): RMSNorm()
    )
    (output_la

In [4]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, trust_remote_code=True)

## 使用微调前 ChatGLM3

In [5]:
input_text = "解释下乾卦是什么？"

In [6]:
response, history = base_model.chat(tokenizer, query=input_text)

In [7]:
print(response)

乾卦是八卦之一，也是八宫图说、易经、易学中常用的符号，它是由两个八卦相组合而成：乾为天，表示刚强、积极、行动力、领导等含义。

乾卦的含义非常丰富，它既表示一种刚强、积极、行动力、领导等含义，也象征一种最高权威、力量、刚毅、坚定、行动、进展、成功等。同时，乾卦也代表着一种宇宙的力量和自然的法则，它反映了天地间万物运行的规律和变化。

在易经中，乾卦的卦辞是“元、亨、利、贞”，这四个字代表了乾卦的基本含义和特质。其中，“元”表示万物之始，“亨”表示通，顺利，“利”表示有利、得利，“贞”表示正、正确。综合起来，乾卦的含义就是：代表宇宙间万物运行的规律和变化，具有刚强、积极、行动力、领导等含义，象征着最高权威、力量、坚定、进展、成功等。


In [8]:
response, history = base_model.chat(tokenizer, query="地水师卦是什么？", history=history)
print(response)

地水师卦是《易经》中的一种卦象，由两个卦相组合而成：地卦和水分开。地卦是八卦之一，代表地球、土地、坚实等含义；水卦则代表水、柔软、润泽等含义。地水师卦的含义丰富，代表了自然、人类和军队之间的关系，象征着团结、众志成城、力量和胜利。

在《易经》中，地水师卦的卦辞是“元、亨、利、贞”，与乾卦的含义类似。综合起来，地水师卦的含义就是：代表自然、人类和军队之间的关系，象征着团结、众志成城、力量和胜利。它反映了天地间万物运行的规律和变化，同时也表达了人们应该顺应自然、团结合作、积极向上的精神。


## 微调前后效果对比

#### 加载 QLoRA Adapter(Epoch=50, Overfit, handmade-dataset)

In [9]:
from peft import PeftModel, PeftConfig

epochs = 50
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}"

config = PeftConfig.from_pretrained(peft_model_path)
model = PeftModel.from_pretrained(base_model, peft_model_path)

### 使用微调后的 ChatGLM3-6B

In [10]:
def compare_chatglm_results(query):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\nChatGLM3-6B微调后：\n{ft_response}")
    return base_response, ft_response

In [11]:
base_response, ft_response = compare_chatglm_results(query="解释下乾卦是什么？")

问题：解释下乾卦是什么？

原始输出：
乾卦是《易经》中的卦名，它是由两个阴爻夹一个阳爻构成，象征着天，表明了天的刚强，如乾果，如雄鸡，如老人。乾卦象征着阳刚之性，表明了积极向上，阳刚之气，它的基本性质是“元”，表示开始，亨通，万事通，象征万事万物的开始和发展。

乾卦的卦象是由三个阳爻夹两个阴爻构成，阴爻代表阴柔之性，阳爻代表阳刚之性。在乾卦中，阴爻夹阳爻，表示阴柔之辈begin;阳爻表示阳刚之性。表示强健有力。乾卦告诉我们，只有阳刚之性和阴柔之性的结合，才能使事物发展成功。

乾卦的含义是：元、亨、利、贞。元表示开始，亨表示通，利表示通，贞表示正。它告诉人们，只有开始，才能通，只有正，才能利。


ChatGLM3-6B微调后：
[gMASK]sop 解释下乾卦是什么？乾卦是乾位的卦象，也是六十四卦中的第一卦。乾卦是由两个阳爻夹一个阴爻构成，象征着天，反映了宇宙的运行和自然的法则。乾卦的卦辞是“元亨，利贞”，意味着大吉大利，符合正道。乾卦的主卦是乾卦，六爻都为阳爻，象征着天，反映了宇宙的运行和自然的法则。乾卦的阳爻代表天、云、云气等，阴爻代表地，表示天上的云彩，寓意着云彩的兴盛和地上的润泽。


In [12]:
base_response, ft_response = compare_chatglm_results(query="地水师卦")

问题：地水师卦

原始输出：
师卦原文：师。贞，丈人吉，无咎。象曰：地中有水，师。君子以容民畜众。白话文解释：师卦象征军队，无灾祸。《象辞》说：下卦为坎（水），上卦为坤（地），如大地容纳江河，君子应容纳众人。《断易天机》解：师卦坤上坎下，象征军众，需德高长者统率以吉无咎。北宋易学家邵雍解：忧劳动众，公正无私排难。得卦者应包容他人，努力排除困难。台湾国学大儒傅佩荣解：时运包容他人，财运有财需珍惜，家宅旧亲联姻吉，身体腹胀调气。传统解卦：异卦（下坎上坤），“师”指军队。坎为水险，坤为地顺，寓兵于农，用兵应顺势，故化凶为吉。


ChatGLM3-6B微调后：
[gMASK]sop 地水师卦 师卦原文：师。贞，丈人吉，无咎。象曰：地中有水，师。君子以容民畜众。白话文解释：师卦象征军队指挥，无灾祸。《象辞》说：下卦为坎（水），上卦为坤（地），如大地容纳江河，君子应容纳众人。《断易天机》解：师卦坤上坎下，象征军众，需德高长者统率以吉无咎。北宋易学家邵雍解：忧劳动众，公正无私排难。得卦者应包容他人，努力排除困难。台湾国学大儒傅佩荣解：时运包容他人，财运有财需珍惜，家宅旧亲联姻吉，身体腹胀调气。传统解卦：异卦（下坎上坤），“师”指军队。坎为水险，坤为地顺，寓兵于农，用兵应顺势，故化凶为吉。


In [13]:
base_response, ft_response = compare_chatglm_results(query="天水讼卦")

问题：天水讼卦

原始输出：
讼卦（卦名：讼）是《易经》中卦名。讼卦象征诉讼，反映诉诸法庭的状况。诉讼是矛盾的焦点，是矛盾双方的权益争议，处理诉讼应当公平公正，既不能曲解法律，也不能循私枉法。应该以理性和法度为依据，以和平的方式解决争端。

讼卦象征 Litigation（诉讼），黄色象征地（代表刚毅），红色象征血（代表危险），象征险阻，凶险。讼卦既指诉讼诉讼诉讼争端，也指因此事带来危险。

 translator：

天水讼卦

天水一卦，刚毅莫测（象征险恶），血盆大德（血的意象）

讼 险症 既济

初六 初六：既济，舍去，利（刚刚截止，有利于避免危险）

解读：
诉讼的纠纷刚刚截止，避免危险，有利于舍去。

六二 两仪既建，险势已去（既指诉讼争议刚止，又指险势消除）

解读：
通过协商成功的避免了诉讼，有利于双方，是好的。

六三 食湿之诉（诉讼如同饮食之诉，无益于事）

解读：
诉讼如同饮食之诉，无益于事，没有必要诉讼，避免诉讼。

六四 既许既 budget（既定既算，预算）

解读：
同意并支付，无不利害。指诉讼成功，可获利益。

六五  reserved（预留），永无（永久）利（无益）

解读：
预留不获利，避免诉讼。

六六 普遍卦象（普遍之卦）

解读：
讼卦卦既指诉讼争端，也指因此事带来危险。卦象表示普遍，指在各种情况下都应避免诉讼，以和平方式解决争端。

以上内容来源于《易经》中的讼卦卦辞。


ChatGLM3-6B微调后：
[gMASK]sop 天水讼卦 讼卦原文：讼。有孚，窒惕，中吉，终凶。利见大人，不利涉大川。象曰：天与水违行，讼。君子以做事谋始。白话文解释：讼卦象征虽有利可图但需警惕。事情初吉后凶，利于见贵人，不宜涉水。《象辞》说：上卦为乾（天），下卦为坎（水），天水相隔，事理不合，君子需慎重谋事。《断易天机》解：讼卦乾上坎下，刚遇险，必有争论，多不吉。北宋易学家邵雍解：天高水深，远离不亲，慎谋退守则无凶。得此卦者，身心不安，多争诉，宜修身养性。台湾国学大儒傅佩荣解：时运受阻，财运初谨慎终获利，家宅君子求淑女，身体预防胜于治疗。传统解卦：异卦（下坎上乾），刚健遇险，彼此反对，生争讼，需慎重戒惧。


#### 加载 QLoRA Adapter(Epoch=3, automade-dataset)

In [14]:
from peft import PeftModel, PeftConfig

epochs = 3
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}"

config = PeftConfig.from_pretrained(peft_model_path)
model = PeftModel.from_pretrained(base_model, peft_model_path)

In [15]:
def compare_chatglm_results(query):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\nChatGLM3-6B (Epoch=3, automade-dataset) 微调后：\n{ft_response}")
    return base_response, ft_response

In [16]:
base_response, ft_response = compare_chatglm_results(query="解释下乾卦是什么？")

问题：解释下乾卦是什么？

原始输出：
乾卦是周易中的一卦，代表天，象征刚健强劲的特性。它由两个卦象叠加而成，上卦是乾卦，下卦是坤卦。在周易中，上卦代表天，下卦代表地，因此乾卦代表天地的力量和特性。

乾卦的卦象是天行健，代表刚健强劲的特性。乾卦由六个爻组成，每个爻都有刚健强劲的特点，由阳爻（乾）和阴爻（坤）相互组合而成。在周易中，阳代表阳，阴代表阴，相互之间有着密切的关系。

乾卦的哲学内涵是刚健不恒，代表天地的力量和特性。它象征刚健强劲的特性，表明了人类面对困难和挑战时，应具备坚忍不拔、勇往直前的精神。同时，乾卦也提示我们，天地的力量是无穷无尽的，需要我们不断求知、进步，以适应不断变化的客观现实。

在周易中，乾卦被认为是最吉利的卦象，预示着卜者将会取得成功。同时，也被认为是一卦之始，代表着开始新的旅程，面对新的挑战。

在解卦时，乾卦预示着大吉大利，表示刚健不恒的特性，预示着成功。在 Business的象征中，乾卦表示 business 的力量和刚健强劲的特性，预示着商业活动将会取得成功。

在解卦时，乾卦提示刚健不恒，表示情况将会发生改变，需要坚忍不变。在决策时，应该坚持坚忍不变的方针，保留阳刚之志， benefit 最终将到来。

总结起来，乾卦代表天地的力量和特性，预示着刚健不恒的状况即将发生改变，提示在面临困难和挑战时，应具备坚忍不拔、勇往直前的精神。


ChatGLM3-6B (Epoch=3, automade-dataset) 微调后：
[gMASK]sop 解释下乾卦是什么？ 乾卦是周易中的一卦，代表天，由六个阳爻组成，象征着刚健强劲的特性。乾卦代表纯阳之象，具有强盛的力量，象征着世界充满活力。在乾卦中，阳爻处于每一卦位，象征着天神的力量遍佈全身，象征着刚健强劲的特性。在乾卦中，阳爻表示贞正，正位，正足，表明了天神对人类的期望和要求。在乾卦中，人们应该参考天神的特性，努力刚健，积极向前，以达到刚健而强劲的生活。

乾卦的哲学内涵是：天行健，君子以自强不息。在乾卦中，天神刚健强劲，积极向前，表示了自强不息的特性。因此，在乾卦中，人们应该以天神为榜样，努力刚健，积极向前，以达到刚健而强劲的生活。


In [17]:
base_response, ft_response = compare_chatglm_results(query="地水师卦是什么？")

问题：地水师卦是什么？

原始输出：
地水师卦是周易中的一卦，由坎卦（水）和坤卦（地）组合而成，代表地中的水流。地水师卦代表一种状况，说明水流在地下，地平如镜，反映出现象山表象。

地水师卦的核心哲学是：流体处于静止状态时，水流在地下，地平如镜，反映出现象山表象。这个卦启示人们，在相处中应该相互谦让，互相包容，如同地水相融，和谐共处。

地水师卦的哲学深度：

地水师卦所反映的是水在地下流动、地平如镜的景象，流体处于静止状态时，水流在地下，地平如镜，反映出现象山表象。这表示流体处于静止状态时，水流在地下，地平如镜，反映出现象山表象。

地水师卦的 interpreted time：

地水师卦所启示的是，在相处中应该相互谦让，互相包容，如同地水相融，和谐共处。这需要在地水师卦的基础上，结合具体的情境和状况，进行灵活应用。

地水师卦的哲学深度和 interpretd time：

地水师卦所反映的是流体处于静止状态时，水流在地下，地平如镜，反映出现象山表象。这表示流体处于静止状态时，水流在地下，地平如镜，反映出现象山表象。地水师卦所启示的是，在相处中应该相互谦让，互相包容，如同地水相融，和谐共处。这需要在地水师卦的基础上，结合具体的情境和状况，进行灵活应用。


ChatGLM3-6B (Epoch=3, automade-dataset) 微调后：
[gMASK]sop 地水师卦是什么？ 地水师卦是一个由坎卦（水）上承坤卦（地）组成的卦象，代表地上的水，表示地上的水都归到坤卦中来，具有养活万物的作用。在卜问过程中，如果卜者如同地上的水归到坤卦中来，就能得到吉祥，并且能够养活万物。

根据《易经》的解卦，地水师卦代表的是泰（太过）卦，表示事情已经过度，需要有所不为，得过且过。泰卦预示着吉祥、顺利，但需要避免过度自信和冒险，保持谦逊和谨慎。

泰卦的哲学启示是：事物发展到一定程度就会过度，需要有所不为，得过且过。过于过度自信和冒险，会带来灾难。因此，在遇到吉祥顺利的事情时，要谨慎谦逊，保持冷静，谨慎行事。

地水师卦的哲学深度：

地水师卦是泰卦的一个变体，表明在泰卦的基础上，要谨慎行事，保持谦逊和谨慎。在吉祥顺利的事情中，要谨慎谦逊，保持冷静，谨慎行事。

地水师卦的启示：

地上的水都归到坤卦中来，具有养活万物的作用。在卜问过程中，如果卜者如同地上的水归到坤卦中来

In [18]:
base_response, ft_response = compare_chatglm_results(query="周易中的讼卦是什么")

问题：周易中的讼卦是什么

原始输出：
在周易中，讼卦是一个极具深意的卦象。上卦为乾（天），下卦为坎（水），两者相背而行，代表天与水违行的状况，象征着事理乖舛和争讼之象。讼卦中有利可图，但必须警惕戒惧，事情中间吉利，但最终会有凶险。在卜卦时，利于会见贵族王公，但不利于涉水渡河。

讼卦的核心哲学是：开始可能顺利，有所收获，但随后会遇到困难和挫折。因此，务必慎之又慎，不得固执已见，避免介入诉讼纠纷的争执之中。退让而不固执，求得化解，安于正理，可免除意外之灾。陷入争讼，即使获胜，最后还得失去，得不偿失。

讼卦的经商指引是：和气生财，吃亏是福，切勿追求不义之财。在商业谈判中要坚持公正、公平、互利的原则，尽量避免发生冲突。

对于决策，讼卦提醒我们，争强好胜，不安于现状，为改变命运和超越他人而奋斗。但缺乏持之以恒的毅力，容易得罪他人，带来诉讼之灾。因此，接受教训，引以为戒，可功成名就。

讼卦所蕴含的智慧是：在面对争端和异见时，要善于退让和求和，坚守正道，谨慎处事，以避免不必要的冲突和损失。


ChatGLM3-6B (Epoch=3, automade-dataset) 微调后：
[gMASK]sop 周易中的讼卦是什么卦象

 讼卦是周易中的一卦，代表诉讼和纷争。它由两个卦象组成，上卦是乾（天），下卦是坎（水）。上卦的卦辞是“元”，下卦的卦辞是“添”，两者相辅相成，代表天地的运行。讼卦的取义是：天（阳）地（阴），阳阴相济，事物自然运行。

讼卦的卦象表示，在诉讼过程中，双方坚持己见，难以达成共识。然而，诉讼过程中双方可能会达成和解，从而使事情得以解决。尽管和解可能存在，但双方仍需警惕再次发生冲突的可能性。

讼卦的核心哲学是：尽管双方在诉讼中可能针锋相对，但最终双方都需要对方，寻求和解，从而使事情得以解决。这提示我们在面对冲突和争议时，需要保持冷静和理智，寻求和解，避免冲突升级。

讼卦 also represents the legal situation, indicating that parties may be engaged in legal disputes. The卦 indicates that both parties are stubborn and difficult to reach a consensus, but ultimately 

#### 加载 QLoRA Adapter(Epoch=3, automade-dataset(fixed))

In [24]:
from peft import PeftModel, PeftConfig

epochs = 3
timestamp = "20240129_113522"
peft_model_path = f"models/{model_name_or_path}-epoch{epochs}-{timestamp}"

config = PeftConfig.from_pretrained(peft_model_path)
model = PeftModel.from_pretrained(base_model, peft_model_path)

In [25]:
def compare_chatglm_results(query):
    base_response, base_history = base_model.chat(tokenizer, query)

    inputs = tokenizer(query, return_tensors="pt").to(0)
    ft_out = model.generate(**inputs, max_new_tokens=512)
    ft_response = tokenizer.decode(ft_out[0], skip_special_tokens=True)
    
    print(f"问题：{query}\n\n原始输出：\n{base_response}\n\n\nChatGLM3-6B(Epoch=3, automade-dataset(fixed))微调后：\n{ft_response}")
    return base_response, ft_response

In [26]:
base_response, ft_response = compare_chatglm_results(query="解释下乾卦是什么？")

问题：解释下乾卦是什么？

原始输出：
{'name': '乾卦是周易中的一卦，由两个乾卦叠加而成，代表天公抖擞，自强不息的意象。在这个卦中，预示着卜者将得到上天的眷顾，事业顺利，生活幸福。同时也预示着保持公正、坚毅、努力和冒险精神，才能获得完全的成功。乾卦的核心哲学是：天公抖擞，自强不息。', 'content': '\n乾卦的象征是天，它代表阳刚之力量，象征刚健、刚健、刚健。乾卦的核心哲学是天公抖擞，自强不息，寓意着君子应当学习天公的刚健，努力自强，不断奋斗。\n\n乾卦的运势如下：\n\n- 初九：险阻重重，务必小心谨慎，保持警惕，自强不息。\n-  nine nine：好运降临，事业顺利，收获丰富，声名远扬。\n- 九二：嘉运即将到来，形势顺利，进展快速。\n- 九三：君子应当像天公一样刚健，自我激励，克服困难。\n- 九四：险阻重重，必须小心谨慎，坚定信念，自强不息。\n- 九五：运势极佳，事业成功，声誉良好。\n- 九六：运势依然顺利，事业、事业、事业'}


ChatGLM3-6B(Epoch=3, automade-dataset(fixed))微调后：
[gMASK]sop 解释下乾卦是什么？ 在周易中，乾卦是六十四卦之首，由六个阳爻组成，象征着天。它所代表的是刚健、健行、刚健不屈的意境。乾卦的核心哲学是：天道刚健，运行不已，君子观此卦象，从而以天为法，自强不息。

乾卦象征天，为大通而至正。得此卦者，名利双收，应把握机会，争取成果。然而，切勿过于骄傲自满，而应保持谦逊、冷静和警惕。在事业、经商、求名等方面，乾卦皆暗示着大吉大利，但也警示着必须坚持正道、修养德行，方能永远亨通。

在婚恋方面，乾卦提示着阳盛阴衰，但也强调刚柔相济，相互补足，形成美满的结果。在决策方面，则是强调刚健、正直、公允，自强不息的实质，需要修养德行、坚定信念，方能克服困难，消除灾难。


In [27]:
base_response, ft_response = compare_chatglm_results(query="地水师卦是什么？")

问题：地水师卦是什么？

原始输出：
{'name': '卜卦是一种古老的占卜方式，可以帮助人们决策。在卜卦中，占卜者会抛三枚硬币，然后根据硬币的正反面来确定卦象。卦象共有64种，其中乾为天，坤为地，震为雷，巽为风，坎为水，离为火，艮为山，兑为泽。占卜者可以根据卦象选择相应的卦辞，以便得到指导。', 'content': '\n地水师卦是一个卦象，它由上卦地，下卦水组成。在这个卦象中，地象征稳定，水象征灵活。根据卦象，占卜者可以得出结论：稳定中寻求灵活，灵活中寻求稳定。地水师卦的卦辞如下：\n\n震卦：雷卦在上，雷动而激起涟漪。\n\n卦象分析：雷代表着刚健，震动，突袭，惊动， tastes否定的声音，君子观此卦象，寓意着行动。震动开始时，君子应当果敢行动。\n\n震卦的卦象由震卦上下组成，象征着果敢的行动。震卦中，上卦为震，象征震动，下卦为兑，象征听到远处传来的消息。整个卦象中，震卦占主导地位，显示了震动的意义。\n\n卦象分析：君子应当果敢行动，因为震动开始时。君子应当果敢行动，但由于震卦占主导地位，因此，震动的意义在整个卦象中占据主导地位。\n\n总结：地水师卦是一个卦象，由地下的稳定性与地上的灵活性组成。占卜者应当在稳定中寻求灵活，灵活中寻求稳定。'}


ChatGLM3-6B(Epoch=3, automade-dataset(fixed))微调后：
[gMASK]sop 地水师卦是什么？ 师卦是一个由坎卦（水）和坤卦（地）相叠而成的异卦。在周易中，这个卦象代表着军队的力量和军情的总指挥，预示着吉祥无灾。象辞中描述了地水师卦的特性和吉祥之兆。

坎卦代表着坎卦（水）的象征，代表着灵活、柔顺、适应变化、团结一致、勇敢顽强、灵活应敌的意思。坤卦代表着坤卦（地）的象征，代表着安顺、温和、柔顺、安泰、团结、忠诚、坚韧不拔的意思。

师卦的核心哲学是：灵活变通，团结一致，勇敢顽强的精神，是战胜困难的法宝。在事业、经商、求名、婚恋等方面的决策中，都需要借助卦象来指导，结合个人喜好和实际情况，积极向前。

总体来说，师卦是一个充满吉祥的卦象，代表着勇敢、智慧和团结，预示着成功和成就。


In [28]:
base_response, ft_response = compare_chatglm_results(query="周易中的讼卦是什么")

问题：周易中的讼卦是什么

原始输出：
在周易中，讼卦是一个充满警示的卦象。它由上卦乾（天）和下卦坎（水）组成，代表着天与水背道而驰，形成争讼的局面。虽然事情开始时有利可图，但必须警惕戒惧，因为中间虽然吉利，但最终会带来凶险。对于涉及大川，涉水渡河的行动不利。因此，君子观此卦象，应当慎之又慎，杜绝争讼之事，并在谋事之初谨慎行事。讼卦的核心哲学是要避免争讼，退而让人，求得化解，安于正理，方可避免意外之灾。在事业上，务必避免介入诉讼纠纷的争执之中，与其这样，不如退而让人。即使最终获胜，也难免得失不均。经商方面，要坚持公正、公平、互利的原则，避免冲突，这样会有好结果。而对于求名、婚恋和决策，也都需要慎重行事，避免盲目追求，退让让人，可助事业、婚姻和决策的发展。


ChatGLM3-6B(Epoch=3, automade-dataset(fixed))微调后：
[gMASK]sop 周易中的讼卦是什么样子 讼卦是一个充满警示的卦象，它由上卦乾（天）和下卦坎（水）组成。在周易中，乾卦代表天，代表着刚强、刚硬、刚毅、刚直，坎卦则代表水，象征着柔顺、退让、谦逊、含蓄。因此，的上卦乾，下卦坎，形成天与水相违的卦象。虽然初始看起来吉祥，但必须警惕，因为中间he，最后 unfavorable。如果能够在行动中保持警惕和谨慎，那么可以带来吉利。然而，必须警惕吉利转瞬即逝，必须谨慎行事以应对各种情况。更多关于周易的占卜和解读，请咨询专业命理师。
